In [11]:
import os

os.chdir('/sidecar_after')

gpu_list = [3]
cuda_device = ','.join([str(i) for i in gpu_list])

print(f'Using GPU: {cuda_device}')

# cuda_device = '1,3'
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

import math

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import traceback
from collections import OrderedDict
from tqdm import tqdm, trange

import random

from tensorboardX import SummaryWriter

from scipy import integrate

sns.set_style("white")
torch.cuda.is_available()


from sidecar import parallel_train
from parameters import params

torch.set_default_dtype(torch.float64)
torch.set_default_device("cuda")

Using GPU: 3


# Parallel training

In [12]:
import torch.multiprocessing as mp

num_processes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

param_vector = [16, 32, 64, 128, 256]



result_matrix = np.zeros((len(param_vector), 10, 14))


i = 0

for width in param_vector:
    
    shared_params = params()

    shared_params.width = width
    shared_params.learning_rate = 0.0001
    shared_params.width_R = 8
    shared_params.num_hidden_R = 1
    shared_params.num_hidden = 2
    shared_params.h_train = 2/256
    shared_params.coe_without_R = 0.0
    shared_params.coe_structure = 10.0
    shared_params.batch_size = 2046
    shared_params.tol = 100.0
    shared_params.epoches_read = 20000
    shared_params.epoches = 10000
    # shared_params.gamma = 0.9999
    # shared_params.Adam_proportion = 0.95

    if __name__ == '__main__':
        mp.set_start_method('spawn', force=True)

        manager = mp.Manager()
        results_queue = manager.Queue()

        num_tasks = num_processes
        num_gpus = len(gpu_list)

        processes = []

        for task_id in num_tasks:
            
            gpu_id = task_id % num_gpus

            print(f"Task {task_id} on GPU {gpu_list[gpu_id]}")
                
            p = mp.Process(target=parallel_train, args=(task_id, shared_params, results_queue, gpu_id))
            p.start()
            processes.append(p)

        for p in processes:
            p.join()
        

        while not results_queue.empty():
            (idx, loss_vector) = results_queue.get()
            result_matrix[i, idx, :] = loss_vector

    i += 1

print(result_matrix)